In [ ]:
!pip install bs4
!pip install requests
!pip install pymongo
!pip install dnspython

     |████████████████████████████████| 245kB 6.3MB/s 


In [ ]:
import requests 
from bs4 import BeautifulSoup
import pandas as pd
import json
from pandas.io.json import json_normalize
import os
import urllib

In [ ]:
#Predição do encoding do arquivo
def predict_encoding(file_path, n_lines=20):
    '''Predict a file's encoding using chardet'''
    import chardet
    with open(file_path, 'rb') as f:
        rawdata = b''.join([f.readline() for _ in range(n_lines)])

In [ ]:
df = pd.DataFrame()

In [ ]:
ano = 10
ultima = 9600
for pagina in range(0, ultima, 100):
  remote_file_name = "http://api.salic.cultura.gov.br/v1/projetos?sort=PRONAC:asc&ano_projeto={}&offset={}&limit=100&format=csv".format(ano, pagina)
  df_file = os.path.join("projetos{}-{}.csv".format(ano, pagina))
  urllib.request.urlretrieve(remote_file_name, df_file) 
  df_csv = pd.read_csv(df_file,sep=',',encoding=predict_encoding(df_file),error_bad_lines=False)
  df = df.append(df_csv) 

In [ ]:
df.to_csv("arquivo_projetos_cultura_20{}.csv".format(ano))

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14152 entries, 0 to 51
Data columns (total 32 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   etapa                  6619 non-null   object 
 1   providencia            14152 non-null  object 
 2   area                   14152 non-null  object 
 3   enquadramento          14152 non-null  object 
 4   objetivos              6623 non-null   object 
 5   ficha_tecnica          6619 non-null   object 
 6   situacao               14152 non-null  object 
 7   outras_fontes          14152 non-null  float64
 8   acessibilidade         6621 non-null   object 
 9   sinopse                6500 non-null   object 
 10  nome                   14152 non-null  object 
 11  cgccpf                 14152 non-null  object 
 12  mecanismo              14152 non-null  object 
 13  segmento               14152 non-null  object 
 14  PRONAC                 14152 non-null  object 
 15  estra

In [ ]:
df[['ano_projeto','valor_captado']].groupby(['ano_projeto']).valor_captado.sum()

ano_projeto
13            1.214718e+09
14            1.259708e+09
14            1.300615e+07
2014-12-17    3.086320e+05
Name: valor_captado, dtype: float64

In [ ]:
df[df['nome'].str.contains('eichler', na=False)]

,etapa,providencia,area,enquadramento,objetivos,ficha_tecnica,situacao,outras_fontes,acessibilidade,sinopse,nome,cgccpf,mecanismo,segmento,PRONAC,estrategia_execucao,valor_aprovado,justificativa,resumo,valor_solicitado,especificacao_tecnica,municipio,data_termino,UF,impacto_ambiental,democratizacao,valor_projeto,proponente,ano_projeto,data_inicio,valor_captado,valor_proposta


In [ ]:
df.groupby(['impacto_ambiental']).PRONAC.count()

impacto_ambiental
(Não se aplica)                                                                                                                                                                                             1
(não se aplica)                                                                                                                                                                                             2
* IMPACTO AMBIENTAL No intuito de minimizar o impacto ambiental negativo, iremos plantar 50 mudas de árvores no Banhado. O Banhado é o cartão postal de São José dos Campos. Com uma imensa área ve         1
* Não se trata de evento ao ar livre. Os eventos aqui previstos serão realizados em teatros fechados.                                                                                                       1
* Panfletos incentivando a organização de lixos e educação para com o mesmo. * lixeiras serão colocadas nos locais abertos. * momento de reflexão para com nos

In [ ]:
df.area.value_counts(normalize=True) * 100

Artes Integradas       53.200961
Artes Cênicas          16.026003
Música                 12.832109
Humanidades             7.730356
Artes Visuais           4.925099
Audiovisual             4.275014
Patrimônio Cultural     1.010458
Name: area, dtype: float64

In [ ]:
df.segmento.value_counts(normalize=True) * 100

Cultura Popular                          53.200961
Teatro                                   11.885246
Música Instrumental                       5.243075
Música Popular Cantada                    5.137083
Exposição de Artes Visuais                4.126625
                                           ...    
Doações de Acervos de Artes Visuais       0.007066
Distribuição Cinematográfica              0.007066
1412064                                   0.007066
Restauração de Acervos                    0.007066
Restauração de Patrimônio Museológico     0.007066
Name: segmento, Length: 63, dtype: float64

In [ ]:
df.groupby(['situacao']).PRONAC.count().sort_values(ascending=False).head(30)

situacao
Edital - Encaminhado a Comissão de Avaliação                                                      2327
Arquivado                                                                                         1947
Arquivado - por excesso de prazo sem captação                                                     1875
Projeto  aceito no edital                                                                         1548
Apresentou prestação de contas                                                                    1396
Edital - Avaliado pela Comissão                                                                   1145
Projeto de edital - encaminhado para avaliação                                                     938
Arquivado - solicitação de arquivamento, de projeto de incentivo fiscal, feito pelo proponente     474
Arquivado - não atendimento à diligência técnica                                                   459
Prestação de Contas Aprovada                                    

In [ ]:
df_prop = pd.DataFrame()

In [ ]:
#proponentes
ultima = 48100
for pagina in range(0, ultima, 100):
  remote_file_name = "http://api.salic.cultura.gov.br/v1/proponentes?&offset={}&limit=100&format=csv".format(pagina)
  df_file = os.path.join("proponentes-{}.csv".format(pagina))
  urllib.request.urlretrieve(remote_file_name, df_file) 
  df_csv = pd.read_csv(df_file,sep=',',encoding=predict_encoding(df_file),error_bad_lines=False)
  df_prop = df_prop.append(df_csv) 

In [ ]:
df_prop.to_csv("arquivo_proponentes_cultura.csv")

In [ ]:
pd.options.display.max_colwidth = 0

In [ ]:
df_prop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48072 entries, 0 to 71
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   nome           48046 non-null  object 
 1   cgccpf         48072 non-null  object 
 2   tipo_pessoa    48072 non-null  object 
 3   responsavel    47254 non-null  object 
 4   UF             48072 non-null  object 
 5   total_captado  48072 non-null  float64
 6   municipio      48057 non-null  object 
dtypes: float64(1), object(6)
memory usage: 2.9+ MB


In [ ]:
df_prop[df_prop['cgccpf'] == '***467288**']

,nome,cgccpf,tipo_pessoa,responsavel,UF,total_captado,municipio
50,Miguel Antônio Testa Barella,***467288**,fisica,Miguel Antônio Testa Barella,SP,0.0,São Paulo
90,Cia da Lua de Tetro Infantil,***467288**,fisica,fabiana consiglio de noronha aguiar,SP,0.0,Santos
11,Peticia Carvalho de Moraes,***467288**,fisica,Peticia Carvalho de Moraes,SP,0.0,São Paulo


In [ ]:
df_prop[df_prop['responsavel'].str.contains('Porchat', na=False)]

,nome,cgccpf,tipo_pessoa,responsavel,UF,total_captado,municipio
21,Fondo Filmes LTDA,09504236000130,juridica,Fábio Porchat de Assis,RJ,0.0,Rio de Janeiro
16,FÁBIO PORCHAT DE ASSIS PRODUÇÕES LTDA-ME,19028036000101,juridica,Fábio Porchat de Assis,SP,224145.0,São Paulo


In [ ]:
df_prop.cgccpf.value_counts().head(10)

***985548**    3
***467288**    3
***233841**    3
***027148**    3
***218626**    3
***762609**    3
***448818**    3
***612643**    3
***130647**    3
***742598**    3
Name: cgccpf, dtype: int64